In [ ]:
import numpy as np
from typing import List, Tuple
from sentence_transformers import SentenceTransformer, CrossEncoder
import faiss
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

In [ ]:
class RAG:
    def __init__(self, documents: List[str], gpt_model: str = "gpt2"):
        """
        Initialize RAG system:
        - Load bi-encoder & cross-encoder
        - Embed documents
        - Build FAISS index
        - Load HuggingFace GPT LLM
        """
        self.documents = documents
        self.doc_ids = list(range(len(documents)))

        print("Loading models…")
        self.embedder = SentenceTransformer("all-MiniLM-L6-v2")
        self.reranker = CrossEncoder("cross-encoder/ms-marco-MiniLM-L6-v2")

        print("Loading HuggingFace LLM:", gpt_model)
        self.tokenizer = AutoTokenizer.from_pretrained(gpt_model)
        self.llm = AutoModelForCausalLM.from_pretrained(gpt_model)

        print("Building embeddings…")
        self.doc_embeddings = self.embedder.encode(
            documents, convert_to_numpy=True, show_progress_bar=True
        )

        dim = self.doc_embeddings.shape[1]

        print("Building FAISS index…")
        self.index = faiss.IndexFlatL2(dim)
        self.index.add(self.doc_embeddings)

        print("RAG System Ready!")

    def retrieve(self, query: str, k: int = 10) -> List[Tuple[int, str]]:
        q_emb = self.embedder.encode(query, convert_to_numpy=True)
        D, I = self.index.search(np.array([q_emb]), k)
        return [(i, self.documents[i]) for i in I[0]]

    def rerank(self, query: str, retrieved_docs: List[Tuple[int, str]]):
        pairs = [(query, doc) for (_, doc) in retrieved_docs]
        scores = self.reranker.predict(pairs)

        reranked = sorted(
            [
                {
                    "doc_id": retrieved_docs[i][0],
                    "text": retrieved_docs[i][1],
                    "score": float(scores[i]),
                }
                for i in range(len(scores))
            ],
            key=lambda x: x["score"],
            reverse=True,
        )
        return reranked

    def build_context(self, reranked_docs, top_n=3) -> str:
        return "\n".join(
            [f"[Doc {d['doc_id']}] {d['text']}" for d in reranked_docs[:top_n]]
        )

    def generate_answer(self, query: str, context: str, max_tokens: int = 150):
        """
        Uses HuggingFace GPT model to generate answer using context.
        """
        prompt = (
            f"Use the context below to answer the question.\n\n"
            f"Context:\n{context}\n\n"
            f"Question: {query}\nAnswer:"
        )

        inputs = self.tokenizer(prompt, return_tensors="pt")
        outputs = self.llm.generate(
            **inputs,
            max_length=max_tokens,
            num_beams=5,
            no_repeat_ngram_size=3,
            early_stopping=True
        )

        return self.tokenizer.decode(outputs[0], skip_special_tokens=True)

    def answer(self, query: str, top_n: int = 3):
        print("\nRetrieving…")
        retrieved = self.retrieve(query)

        print("Reranking…")
        reranked = self.rerank(query, retrieved)

        print("\nTop Relevant Docs:")
        for d in reranked[:top_n]:
            print(f"- (Score {d['score']:.3f}) Doc {d['doc_id']}: {d['text']}")

        context = self.build_context(reranked, top_n)

        print("\nGenerating LLM Answer…")
        llm_answer = self.generate_answer(query, context)

        print("\nFinal Answer:")
        print(llm_answer)

        return llm_answer



if __name__ == "__main__":
    # Example documents
    docs = [
        "Berlin had a population of 3,520,031 registered inhabitants.",
        "Berlin has about 135 million day visitors per year.",
        "Around 600,000 Berliners are registered in sports clubs.",
        "Paris is the capital of France with about 2.1 million people.",
        "Tokyo has a population exceeding 13 million residents.",
    ]

    rag = RAG(docs, gpt_model="EleutherAI/gpt-neo-1.3B")

    query = "Total population of Berlin?"

    rag.answer(query)

🔄 Loading models…
📦 Loading HuggingFace LLM: EleutherAI/gpt-neo-1.3B
📌 Building embeddings…


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

📌 Building FAISS index…
✅ RAG System Ready!


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



🔍 Retrieving…
📊 Reranking…

📘 Top Relevant Docs:
- (Score 9.322) Doc 0: Berlin had a population of 3,520,031 registered inhabitants.
- (Score 0.716) Doc 1: Berlin has about 135 million day visitors per year.
- (Score 0.383) Doc 2: Around 600,000 Berliners are registered in sports clubs.

🤖 Generating LLM Answer…

🧠 Final Answer:
Use the context below to answer the question.

Context:
[Doc 0] Berlin had a population of 3,520,031 registered inhabitants.
[Doc 1] Berlin has about 135 million day visitors per year.
[Doc 2] Around 600,000 Berliners are registered in sports clubs.

Question: Total population of Berlin?
Answer: Berlin has a total population of about 3,500,000.
Question 2: How many sports clubs are there in Berlin? 
Answer 2: There are around 600 sports clubs in Berlin. 
Question 3: What is the average age of a sports club member?  

Answer 3: The average age for a member of a sport club in Berlin
